# Importing Libraries

In [2]:
import sys
import os
import json
import numpy as np
from tqdm import tqdm
import torch
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils_data import  generateTargetDataBuySide, generateTargetDataSellSide, getTechnicalIndicators, normalize_dataframe_with_mean_std
from utils_data import UpstoxStockDataFetcher

# Load data

In [27]:
fetcher = UpstoxStockDataFetcher()
stock_symbol = "SUZLON"
start_date = "2025-06-09"
end_date = "2025-06-17"
df = fetcher.get_stock_data(stock_symbol, start_date, end_date)
df


,time,open,high,low,close,volume
0,2025-06-09 09:15:00+05:30,67.75,68.21,67.00,67.65,11799029
1,2025-06-09 09:16:00+05:30,67.65,67.65,67.01,67.45,5551341
2,2025-06-09 09:17:00+05:30,67.45,67.85,67.39,67.70,2244711
3,2025-06-09 09:18:00+05:30,67.69,68.07,67.64,68.00,3492135
4,2025-06-09 09:19:00+05:30,68.00,68.30,67.88,68.10,3869514
...,...,...,...,...,...,...
2620,2025-06-17 15:25:00+05:30,64.62,64.62,64.60,64.61,100057
2621,2025-06-17 15:26:00+05:30,64.60,64.61,64.59,64.60,334285
2622,2025-06-17 15:27:00+05:30,64.61,64.66,64.46,64.65,592029
2623,2025-06-17 15:28:00+05:30,64.65,64.66,64.60,64.66,203337


In [28]:
df = getTechnicalIndicators(df)
target_buy = generateTargetDataBuySide(df,1.005,0.99)
target_sell = generateTargetDataSellSide(df,0.995,1.01)


Processing Sell Side Data: 100%|██████████| 2598/2598 [00:58<00:00, 44.63it/s] 


In [29]:
df_normalized, norm_param = normalize_dataframe_with_mean_std(df)

c:\Users\srija\Assignment\Trading\utils_data.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540
c:\Users\srija\Assignment\Trading\utils_data.py:207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '42' has dtype incompatible with datetime64[ns, UTC+05:30], please explicitly cast to a compatible dtype first.
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540


In [30]:
def get_state(df, current_step):
    row = df.iloc[current_step]
    state = np.array([
        row['time'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['MA50'],
        row['RSI'],
        row['MACD'],
        row['BB_upper'],
        row['BB_lower'],
        row['ADX'],
        row['CCI'],
        row['ATR'],
        row['ROC'],
        row['OBV']
    ], dtype=np.float32)
    return state

# Loading Model

In [31]:
from Models.DQN import DQN,DQNAgent
from trading_environment import StockTradingEnv

policy_net = DQN(16, 3)
target_net = DQN(16, 3)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

policy_net.to(device)
target_net.to(device)

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [39]:
model_path = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june\suzlon_14_june_5501.pth"
policy_net.load_state_dict(torch.load(model_path))
policy_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

# Evaluate Model

In [40]:
import time
correct = 0
wrong = 0
blunder =0
latency_list = []
for i in range(0,len(df_normalized)):
    t1 = time.time()
    state = get_state(df_normalized,i)
    with torch.no_grad():
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        q_values = policy_net(state)
        action=q_values.argmax().item()
    ideal_action = 0
    if target_buy.iloc[i]['action'] == "Target Hit":
        ideal_action = 1
    elif target_sell.iloc[i]['action'] == "Target Hit":
        ideal_action = 2
    if action==ideal_action:
        correct += 1
    elif action == 1 and ideal_action ==2:
        blunder += 1
    elif action == 2 and ideal_action ==1:
        blunder += 1    
    else:
        wrong += 1
    t2 = time.time()
    latency = t2 - t1
    latency_list.append(latency)

print(f"Correct: {correct}, Wrong: {wrong}, Blunder: {blunder}")  
print(f"Average Latency: {np.mean(latency_list)} seconds")  

Correct: 751, Wrong: 1538, Blunder: 309
Average Latency: 0.0008870003496893926 seconds
